# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from pprint import pprint

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.2906,-156.7887,-27.99,76,100,2.57,US,1675612729
1,1,mataura,-46.1927,168.8643,14.83,75,99,3.87,NZ,1675612729
2,2,kapaa,22.0752,-159.3190,22.18,90,100,4.12,US,1675612730
3,3,la ronge,55.1001,-105.2842,-4.86,93,100,3.60,CA,1675612730
4,4,ushtobe,45.2514,77.9838,0.43,98,100,1.80,KZ,1675612731


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing cities from city_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [72]:
# Narrow down cities that fit criteria and drop any results with null values

# Set my ideal weather conditions to filter the data
temp_min = 20
temp_max = 35
win_speed_max = 5
cloudiness_max = 30
humidity_max = 50

# Filter the city_data_df DataFrame for the ideal weather conditions
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= temp_min) &
                                   (city_data_df["Max Temp"] <= temp_max) &
                                   (city_data_df["Wind Speed"] <= win_speed_max) &
                                   (city_data_df["Cloudiness"] <= cloudiness_max) &
                                   (city_data_df["Humidity"] <= humidity_max)]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,phalodi,27.1333,72.3667,22.38,21,0,4.54,IN,1675612735
90,90,tenenkou,14.4572,-4.9169,34.27,8,4,4.21,ML,1675612773
165,165,moron,-34.6534,-58.6198,30.57,44,0,1.77,AR,1675612800
172,172,shwebo,22.5667,95.7000,21.93,31,29,2.48,MM,1675612835
181,181,gangapur,26.4833,76.7167,20.02,33,0,1.95,IN,1675612841
208,208,coahuayana,18.7333,-103.6833,28.94,31,0,0.80,MX,1675612852
212,212,merta,26.6500,74.0333,20.43,29,0,1.91,IN,1675612853
219,219,pochutla,15.7432,-96.4661,23.90,43,0,1.99,MX,1675612856
221,221,bucerias,20.7500,-105.3333,24.97,49,4,1.75,MX,1675612857
235,235,puerto escondido,15.8500,-97.0667,28.52,47,0,2.33,MX,1675612818


### Step 3: Create a new DataFrame called `hotel_df`.

In [73]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,phalodi,IN,27.1333,72.3667,21,
90,tenenkou,ML,14.4572,-4.9169,8,
165,moron,AR,-34.6534,-58.6198,44,
172,shwebo,MM,22.5667,95.7000,31,
181,gangapur,IN,26.4833,76.7167,33,
208,coahuayana,MX,18.7333,-103.6833,31,
212,merta,IN,26.6500,74.0333,29,
219,pochutla,MX,15.7432,-96.4661,43,
221,bucerias,MX,20.7500,-105.3333,49,
235,puerto escondido,MX,15.8500,-97.0667,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()

        # Note: I had used pprint to display the response to confirm details
        # pprint(name_address_json)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
phalodi - nearest hotel: LAL NIWAS
tenenkou - nearest hotel: No hotel found
moron - nearest hotel: La Esquina
shwebo - nearest hotel: Pyi Shwe Theingha
gangapur - nearest hotel: No hotel found
coahuayana - nearest hotel: No hotel found
merta - nearest hotel: No hotel found
pochutla - nearest hotel: Posada San Jose
bucerias - nearest hotel: Royal Decameron Complex
puerto escondido - nearest hotel: Hotel Barlovento
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
mazatlan - nearest hotel: Mona Inn
balotra - nearest hotel: No hotel found
areka - nearest hotel: Selam Hotel
santa maria da vitoria - nearest hotel: Hotel Passarela
neuquen - nearest hotel: Hotel Argentina 307
valdivia - nearest hotel: Esmeralda
itapetinga - nearest hotel: Hotel Lacerda
vitoria da conquista - nearest hotel: Faixa Hotel
yangambi - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
aswan - nearest hotel: Yaseen hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
18,phalodi,IN,27.1333,72.3667,21,LAL NIWAS
90,tenenkou,ML,14.4572,-4.9169,8,No hotel found
165,moron,AR,-34.6534,-58.6198,44,La Esquina
172,shwebo,MM,22.5667,95.7000,31,Pyi Shwe Theingha
181,gangapur,IN,26.4833,76.7167,33,No hotel found
208,coahuayana,MX,18.7333,-103.6833,31,No hotel found
212,merta,IN,26.6500,74.0333,29,No hotel found
219,pochutla,MX,15.7432,-96.4661,43,Posada San Jose
221,bucerias,MX,20.7500,-105.3333,49,Royal Decameron Complex
235,puerto escondido,MX,15.8500,-97.0667,47,Hotel Barlovento


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2.0,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing ideal cities for vacation",
    xlabel = "Latitude",
    ylabel = "Longitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)